In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_cell_range(0, IPython.notebook.get_selected_index()+1);
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [2]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

__file__ = 'exercises-geometry.ipynb' 

/home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import math

class Point:
    def __init__(self, x,y=None):
        self.x =float(x)
        self.y =float(y)
    
    def normal(self):
        return Point(-self.y, self.x)
    
    def normalize(self, d=0):
        if d == 0:
            d = math.sqrt(self.x*self.x +self.y*self.y)
        return Point(self.x/d, self.y/d)
    
    def scalar_product(self,other: 'Point'):
        return self.x*other.x + self.y*other.y
    
    def is_orthogonal(self, other: 'Point'):
        product =self.scalar_product(other)
        return math.isclose(product,0)
    
    def is_collinear(self, other: 'Point'):
        return self.is_orthogonal(other.normal())
       
    def __add__(self, other: 'Point'):
        r = NotImplemented
        if isinstance(other, Point):
            r = Point(self.x+other.x, self.y+other.y)
        return r
    
    def __sub__(self, other: 'Point'):
        r = NotImplemented
        if isinstance(other, Point):
            r = Point(self.x-+other.x, self.y-other.y)
        return r
    
    
    def __mul__(self,other):
        r = NotImplemented
        if isinstance(other, float) or isinstance(other, int):
            factor = float(other)
            r = Point(self.x*factor, self.y*factor)
        return r

    def __eq__(self, other: 'Point'):
        r = NotImplemented
        if isinstance(other, Point):
            r = math.isclose(self.x,other.x,abs_tol=1e-9) and  math.isclose(self.y,other.y,abs_tol=1e-9)
        return r
    
    def __repr__(self):
        return f'p({self.x}, {self.y})'
    
    @staticmethod
    def distance(a:'Point', b: 'Point') -> float:
        dx = a.x-b.x
        dy = a.y-b.y
        return math.sqrt(dx*dx + dy*dy)

In [4]:
    
        
class Segment:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
class Line:
    def __init__(self, point: Point, vector: Point):
        self.point = point
        self.vector = vector.normalize()
        
    def contains(self, point: Point):
        a = self.point
        b = point
        v_ab = Point(a.x -b.x, a.y - b.y)
        return v_ab.is_collinear(self.vector)
    
    def intersection(self, line: 'Line'):
        v0 = self.vector
        p0 = self.point
        
        v1 = line.vector
        p1 = line.point
        
        v1_v0 = v1.scalar_product(v0)
        
        dp = p1 - p0
        
        dp_vo = dp.scalar_product(v0)
        dp_v1 = dp.scalar_product(v1)
        
        if math.isclose(v1_v0,1.) :
            raise ValueError('Lines are parallel')
        else: 
            coef0 = ( dp_vo - dp_v1*v1_v0)/(1 - v1_v0*v1_v0)
        
        return v0*coef0 + p0
    
    
    def __repr__(self):
        return f'line({self.point}, {self.vector})'

In [5]:
            
class CircularArc:
    def __init__(self, start: Point, end: Point, tangent: Point):
        self.start = start
        self.end = end
        self.center = CircularArc.compute_center_with_tangent(start, end, tangent)
        self.radius = Point.distance(self.center, self.start)
        self.angle = 2 * math.asin(Point.distance(start, end) / (2* self.radius))
        self.length = self.angle * math.pi * self.radius
    
    @staticmethod
    def compute_center(start, end, radius, angle):
        chord = Point((end.x - start.x), (end.y - start.y))
        c = Point((end.x + start.x)/2, (end.y + start.y)/2)
        ortho_c = chord.normal()
        ortho_c = ortho_c.normalize()
        center = ortho_c*radius*math.cos(angle/2) + c
        return center
    
    @staticmethod
    def compute_center_with_tangent(start, end, tangent):
        ortho_tgt = tangent.normal()
        line_radial = Line(start, ortho_tgt)
        
        chord = end - start
        
        c = Point((end.x + start.x)/2, (end.y + start.y)/2)
        ortho_c = chord.normal().normalize()
        
        line_bissect = Line(c, ortho_c)
        
        try:
            center = line_radial.intersection(line_bissect)
        except ValueError:
            center = c
        return center    
            
    @staticmethod
    def find_angle_and_chord_vector(start, end, tangent):
        a = start
        b = end
        v = tangent
        
        distance = Point.distance(a, b)
        u = Point(b.x - a.x, b.y - a.y)
        u = u.normalize(distance)
        v = v.normalize()
        angle = math.acos(u.scalar_product(v))
        sign = u.x*v.y + u.y*v.x
        angle = 2 * angle * sign
        
        return angle, u, distance

In [6]:
import pixiedust

Pixiedust database opened successfully


In [11]:
%%pixie_debugger
candidat = CircularArc(Point(-1,0), 
                       Point(1,0), 
                       Point(0,1))
assert candidat.radius == 1
assert candidat.angle == math.pi
assert candidat.center == Point(0.,0.)

In [6]:
%%run_pytest[clean] -qq
import pytest

def test_add():
    a = Point(0.5, 0.5)
    b = Point(-1, 1)
    c = a + b
    assert c.x == -0.5
    assert c.y == 1.5
    
def test_equal():
    a = Point(0.5, 0.5)
    b = Point(0.5*(1+1e-10), 0.5)
    assert a == b
    
def test_equal_float_int():
    a = Point(4, 1)
    b = Point(4.0, 1.0)
    assert a == b
    
    
def test_equal_10e17():
    a = Point(0.0, 6.123233995736766e-17)
    b = Point(0.,0.)
    assert a == b

def test_are_orthogonal():
    a = Point(0.5, 0.5)
    b = Point(-1, 1)
    assert a.is_orthogonal(b)
    
def test_are_not_orthogonal():
    a = Point(1, 0.7)
    b = Point(0.5, 1)
    assert not a.is_orthogonal(b)
    
def test_are_collinear():
    a = Point(0.5, 0.5)
    b = Point(1, 1)
    assert a.is_collinear(b)
    

def test_coordinate_belongs_to_line():
    A = Point(1,2)
    line = Line(A, Point(1,1))
    assert line.contains( Point(2,3))
    
def test_intersection():
    line_a = Line(Point(0,0), Point(1,0))
    line_b = Line(Point(0,0), Point(0,1))
    assert line_a.intersection(line_b) == Point(0,0)
    
def test_intersection_2():
    line_a = Line(Point(3,1), Point(1,1))
    line_b = Line(Point(0,2), Point(1,0))
    intersection = line_a.intersection(line_b)
    expected = Point(4,2)
    assert intersection == expected, f'{intersection} != {expected}'  
    
    
def test_intersection_of_parallel_lines():
    line_a = Line(Point(0,1), Point(0,1))
    line_b = Line(Point(1,1), Point(0,1))

    with pytest.raises(ValueError):
        intersection = line_a.intersection(line_b)
    
def test_distance_a_b():
    assert math.isclose(Point.distance(Point(1,0), Point(0,1)), math.sqrt(2))
    
def test_circular_arc():
    arc = CircularArc(start=Point(1,0), end=Point(0,1), tangent=Point(0,1))
    assert math.isclose(arc.angle,(math.pi/2.0))
    

table_vectors_angles = [
    ((-1,0), (1,0), math.pi),
    ((1,0), (0,1), math.pi/2),
    ((1,0), (0,-1), math.pi/2)

]
@pytest.mark.parametrize('a, b, angle', table_vectors_angles)
def test_find_angle_from_2_vectors(a, b, angle):
    def f(u,v):
        return math.acos(u.scalar_product(v))
        
    assert f(Point(a[0],a[1]), Point(b[0],b[1])) == angle
    

................                                                                                                                                                                                            [100%]


In [7]:
%%run_pytest[clean] -qq

def test_find_find_angle_and_chord_vector():
    angle, u, distance = CircularArc.find_angle_and_chord_vector(Point(-1,0), 
                                                 Point(1,0), 
                                                 Point(0,1))    
    assert angle == math.pi
    
def test_simple_arc():
    candidat = CircularArc(Point(-1,0), 
                           Point(1,0), 
                           Point(0,1))
    assert candidat.radius == 1
    assert candidat.angle == math.pi
    assert candidat.center == Point(0.,0.)
    
def test_45deg_arc():
    candidat = CircularArc(Point(1,0), 
                           Point(0,1), 
                           Point(0,1))
    
    assert math.isclose(candidat.angle, math.pi/2)
    assert candidat.center == Point(0,0)
    assert math.isclose(candidat.radius,1)    
    
def test_reverse_45deg_arc():
    candidat = CircularArc(Point(1,0), 
                           Point(0,1), 
                           Point(0,-1))
    
    assert candidat.center == Point(0,0)    
    assert math.isclose(candidat.radius,1)    
    assert math.isclose(candidat.angle, math.pi/2)    

....                                                                                                                                                                                                        [100%]
